In [42]:
from random import randint

M = 1000

lst = [randint(1, 1001) for _ in range(M)]

In [33]:
def first_approach_function(lst):
    sorted_lst = sorted(lst)
    counter = 0
    max_counter = 0
    prev_elem = lst[0]
    for elem in sorted_lst[1:]:
        if elem != prev_elem:
            if counter > max_counter:
                max_counter = counter
                most_frequent_elem = prev_elem
            counter = 0
        counter += 1
        prev_elem = elem
    return most_frequent_elem

In [25]:
def second_approach_function(lst):
    frequency_dict = {}
    for elem in lst:
        corresponding_counter = frequency_dict.get(elem)
        if corresponding_counter is None:
            frequency_dict[elem] = 1
        else:
            frequency_dict[elem] += 1
    return max(frequency_dict, key=lambda key : frequency_dict[key])

In [41]:
import numpy as np

def third_approach_function(lst):
    np_lst = np.array(lst)
    frequency_lst = np.bincount(np_lst)
    return (np.where(frequency_lst == max(frequency_lst))[0][0])